Required Libraries:

In [1]:
from itertools import groupby
import requests
import zipfile
import os
from io import BytesIO
import pandas as pd
import glob

PART I

Download data for years 1990 - 2021 on football teams scores
Extract Data
1. Download zip file that contains multiple subfolders with the scv files
2. Extract zip file content
3. Remove zip file after
4. Read multiple scv files into one dataframe


In [2]:
link1 = "https://aicore-files.s3.amazonaws.com/Data-Science/Football.zip"
req=requests.get(link1)
filename=link1.split('/')[-1]
with open(filename,'wb') as output_file:
    output_file.write(req.content)

with zipfile.ZipFile(filename,"r") as zip_ref:     
    zip_ref.extractall()

os.remove(filename)

path = "./Results"
csv_files = glob.glob(path + "/**/*.csv", recursive = True)
results_df = [pd.read_csv(f) for f in csv_files]
pd.set_option('display.max_columns', None)
score_df  = pd.concat(results_df, ignore_index=True)
score_df

,Home_Team,Away_Team,Result,Link,Season,Round,League,Unnamed: 0
0,Eibar,Celta,0-0,https://www.besoccer.com/match/eibar/celta/202...,2021,1,primera_division,NaN
1,Granada,Athletic,2-0,https://www.besoccer.com/match/granada/athleti...,2021,1,primera_division,NaN
2,Cádiz,Osasuna,0-2,https://www.besoccer.com/match/cadiz/osasuna/2...,2021,1,primera_division,NaN
3,Alavés,Real Betis,0-1,https://www.besoccer.com/match/alaves/betis/20...,2021,1,primera_division,NaN
4,Real Valladolid,R. Sociedad,1-1,https://www.besoccer.com/match/valladolid/real...,2021,1,primera_division,NaN
...,...,...,...,...,...,...,...,...
146636,RBC Roosendaal,Dordrecht,3-2,https://www.besoccer.com/match/rbc/fc-dordrech...,2011,34,eerste_divisie,301.0
146637,Cambuur,PEC Zwolle,3-3,https://www.besoccer.com/match/sc-cambuur-leeu...,2011,34,eerste_divisie,302.0
146638,FC Volendam,BV Veendam,2-2,https://www.besoccer.com/match/fc-volendam/vee...,2011,34,eerste_divisie,303.0
146639,Den Bosch,Almere City,1-0,https://www.besoccer.com/match/fc-den-bosch/al...,2011,34,eerste_divisie,304.0


In [3]:
#Display dataframe information
score_df.info()

<bound method DataFrame.info of               Home_Team        Away_Team Result  \
0                 Eibar            Celta    0-0   
1               Granada         Athletic    2-0   
2                 Cádiz          Osasuna    0-2   
3                Alavés       Real Betis    0-1   
4       Real Valladolid      R. Sociedad    1-1   
...                 ...              ...    ...   
146636   RBC Roosendaal        Dordrecht    3-2   
146637          Cambuur       PEC Zwolle    3-3   
146638      FC Volendam      BV Veendam     2-2   
146639        Den Bosch      Almere City    1-0   
146640  Go Ahead Eagles  AGOVV Apeldoorn    2-1   

                                                     Link Season Round  \
0       https://www.besoccer.com/match/eibar/celta/202...   2021     1   
1       https://www.besoccer.com/match/granada/athleti...   2021     1   
2       https://www.besoccer.com/match/cadiz/osasuna/2...   2021     1   
3       https://www.besoccer.com/match/alaves/betis/20...  

CLEANING DATA
1. Change data types for League, Home_Team, Away_Team into category.
2. Split Result in form #:# into two columns representing Home Team score and Away Team score.
3. Drop columns: Result and extra.
4. Drop rows with NA data (see notes).
5. Change data types into int for Season (year), Round, and both score columns.

Notes:
1. There were only 100 rows of NA data, that has been removed. Not significant.
2. DF used memory size droped from 9+MB to 6.3MB


In [4]:
cols=['League', 'Home_Team','Away_Team']
score_df[cols]=score_df[cols].astype('category')

score_df['Result']=score_df['Result'].astype('string')
score_df[['Home_Score','Away_Score']]=score_df['Result'].str.split('-', expand=True)
score_df['Home_Score'] = pd.to_numeric(score_df['Home_Score'], errors='coerce')
score_df['Away_Score'] = pd.to_numeric(score_df['Away_Score'], errors='coerce')

score_df.drop(['Unnamed: 0', 'Result'], inplace=True, axis=1)
score_df = score_df.dropna()
cols = ['Season', 'Round', 'Home_Score','Away_Score']
score_df[cols]=score_df[cols].astype('int')
score_df

/tmp/ipykernel_28599/1303400552.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df[cols]=score_df[cols].astype('int')


,Home_Team,Away_Team,Link,Season,Round,League,Home_Score,Away_Score
0,Eibar,Celta,https://www.besoccer.com/match/eibar/celta/202...,2021,1,primera_division,0,0
1,Granada,Athletic,https://www.besoccer.com/match/granada/athleti...,2021,1,primera_division,2,0
2,Cádiz,Osasuna,https://www.besoccer.com/match/cadiz/osasuna/2...,2021,1,primera_division,0,2
3,Alavés,Real Betis,https://www.besoccer.com/match/alaves/betis/20...,2021,1,primera_division,0,1
4,Real Valladolid,R. Sociedad,https://www.besoccer.com/match/valladolid/real...,2021,1,primera_division,1,1
...,...,...,...,...,...,...,...,...
146636,RBC Roosendaal,Dordrecht,https://www.besoccer.com/match/rbc/fc-dordrech...,2011,34,eerste_divisie,3,2
146637,Cambuur,PEC Zwolle,https://www.besoccer.com/match/sc-cambuur-leeu...,2011,34,eerste_divisie,3,3
146638,FC Volendam,BV Veendam,https://www.besoccer.com/match/fc-volendam/vee...,2011,34,eerste_divisie,2,2
146639,Den Bosch,Almere City,https://www.besoccer.com/match/fc-den-bosch/al...,2011,34,eerste_divisie,1,0


In [5]:
#memory droped from 9+MB to 6.3MB
score_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146541 entries, 0 to 146640
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   Home_Team   146541 non-null  category
 1   Away_Team   146541 non-null  category
 2   Link        146541 non-null  object  
 3   Season      146541 non-null  int64   
 4   Round       146541 non-null  int64   
 5   League      146541 non-null  category
 6   Home_Score  146541 non-null  int64   
 7   Away_Score  146541 non-null  int64   
dtypes: category(3), int64(4), object(1)
memory usage: 7.4+ MB


In [6]:
#How many counts per League appeared in data
count1 = score_df.groupby('League')['Home_Team'].count()
print(count1)

League
2_liga              10027
bundesliga           9812
championship         9321
eerste_divisie       6485
eredivisie           9671
ligue_1             11629
ligue_2             10888
premier_league      12416
primeira_liga        9265
primera_division    12244
segunda_division    14038
segunda_liga         7988
serie_a              9365
serie_b             13392
Name: Home_Team, dtype: int64


Let's create the dataframe with the data on total number of teams per league

In [7]:
#Number of unique teams per League
s1 = score_df.groupby('League')['Home_Team'].nunique()
s2 = score_df.groupby('League')['Away_Team'].nunique()

df_league =s1.to_frame(name = 'Home_Team').join(s2.to_frame(name='Away_Team'))
df_league

,Home_Team,Away_Team
League,,
2_liga,83,83
bundesliga,46,46
championship,63,63
eerste_divisie,38,38
eredivisie,32,32
ligue_1,49,49
ligue_2,75,75
premier_league,52,52
primeira_liga,43,43


Next step in analysis is creating a separate dataframe with number of wins/draw/loses for home team each year 
1. Add new column Wins with the values: 1 for win, 0 for draw and -1 for lose
2. New dataframe yearly_trend consists of Season, total games, total wins/draws/loses for home team 

In [8]:

score_df.loc[score_df['Home_Score']==score_df['Away_Score'], 'Wins']=0
score_df.loc[score_df['Home_Score']>score_df['Away_Score'], 'Wins']=1
score_df.loc[score_df['Home_Score']<score_df['Away_Score'], 'Wins']=-1
score_df['Wins']= score_df['Wins'].astype('int')

yearly_trend = pd.DataFrame(
    {
        'total':score_df.groupby('Season')['Wins'].size(),
        'wins' : score_df[score_df['Wins']==1].groupby('Season')['Wins'].size(), 
        'losses' : score_df[score_df['Wins']==-1].groupby('Season')['Wins'].size(),
        'draw' : score_df[score_df['Wins']==0].groupby('Season')['Wins'].size()
    })
yearly_trend

/tmp/ipykernel_28599/2263193985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df.loc[score_df['Home_Score']==score_df['Away_Score'], 'Wins']=0
/tmp/ipykernel_28599/2263193985.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['Wins']= score_df['Wins'].astype('int')


,total,wins,losses,draw
Season,,,,
1990,4056,1981,814,1261
1991,4130,1978,792,1360
1992,3964,1885,848,1231
1993,4310,2050,971,1289
1994,4600,2223,995,1382
1995,4526,2145,1018,1363
1996,4526,2143,1058,1325
1997,4830,2258,1169,1403
1998,5064,2447,1195,1422


PART II

Download new file with the match data: season, red cards, yellow cards, per team (home or away)

In [9]:
link2= 'https://aicore-files.s3.amazonaws.com/Data-Science/Match_Info.csv'
match_df = pd.read_csv(link2)
pd.set_option('display.max_columns', None)
match_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143348 entries, 0 to 143347
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Link         143348 non-null  object 
 1   Date_New     143348 non-null  object 
 2   Referee      143348 non-null  object 
 3   Home_Yellow  122798 non-null  float64
 4   Home_Red     122798 non-null  float64
 5   Away_Yellow  122798 non-null  float64
 6   Away_Red     122798 non-null  float64
dtypes: float64(4), object(3)
memory usage: 7.7+ MB


Cleaning Data:
1. Strip the Referee data to the referees names only and make it a category (set number of referees only).
2. Split data columns to only year values to turn into Season.
3. Dropped unnecessary columns: the temp ones and Link.
4. Changed data type for the yellow and red cards into integers and dropped NW values (see notes).

Notes:
1. After data cleaning DF size dropped from 7.7MB+ to 6.5MB. 
2. There were 20550 rows with NA values, which is 14.3%. NA values would not be beneficial for the analysis, therefore the values were removed.

In [10]:
match_df['Referee'] = match_df['Referee'].str.strip('\r\nReferee:')
match_df['Referee'] = match_df['Referee'].astype('category')

match_df['Date_New'] = match_df['Date_New'].astype('string')
match_df['Season1']=match_df['Date_New'].str.split(',', expand=True)[1]
match_df['Season']=match_df['Season1'].str.split(' ', expand=True)[3]

cols=['Link', 'Season1', 'Date_New']
match_df.drop(cols,inplace=True, axis=1)

cols=['Home_Yellow', 'Away_Yellow', 'Home_Red', 'Away_Red', 'Season']
match_df[cols] = match_df[cols].astype('Int64')

match_df=match_df[match_df['Home_Yellow'].isna()==False]

match_df

,Referee,Home_Yellow,Home_Red,Away_Yellow,Away_Red,Season
0,Hans-Jürgen Web,0,0,3,0,1989
1,Kurt Wittk,1,0,0,0,1989
2,Werner Föckl,3,0,2,0,1989
3,Heinz Wern,2,0,2,0,1989
4,Hans-Peter Dellwing,1,0,1,0,1989
...,...,...,...,...,...,...
143343,Anthony Leak,1,0,0,0,1998
143344,Jacques Poulain,0,0,0,0,1994
143345,Kevin Lynch,2,0,1,0,1999
143346,Baert A.,0,0,2,0,2021


In [11]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122798 entries, 0 to 143347
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   Referee      122798 non-null  category
 1   Home_Yellow  122798 non-null  Int64   
 2   Home_Red     122798 non-null  Int64   
 3   Away_Yellow  122798 non-null  Int64   
 4   Away_Red     122798 non-null  Int64   
 5   Season       122798 non-null  Int64   
dtypes: Int64(5), category(1)
memory usage: 6.5 MB


PART III

Download new file with stadium per team per country

In [12]:
link3='https://aicore-files.s3.amazonaws.com/Data-Science/Team_Info.csv'
stadium_df = pd.read_csv(link3)
pd.set_option('display.max_columns', None)
stadium_df

,Team,City,Country,Stadium,Capacity,Pitch
0,Wattenscheid 09,Bochum-Wattenscheid,Germany,Lohrheidestadion,16233,Natural
1,Hertha BSC,Berlín,Germany,Olympiastadion Berlin,76065,Natural
2,Unterhaching,Unterhaching,Germany,Sportpark Unterhaching,15053,Natural
3,Fortuna Köln,Cologne,Germany,Südstadion,14944,Natural
4,MSV Duisburg,Duisburgo,Germany,Schauinsland-Reisen-Arena,31514,Natural
...,...,...,...,...,...,...
539,Latina,Latina,Italy,Stadio Comunale Domenico Francioni,8000,Natural
540,Virtus Entella,Chiavari,Italy,Stadio Comunale Chiavari,4154,Césped Artificial
541,Nuova Cosenza,Cosenza,Italy,Stadio San Vito,24209,Natural
542,Pordenone,Pordenone,Italy,NaN,3089,NaN


DATA CLEANING
1. Country and Pitch is set as a categorical data
2. Team, Stadium, City set as strings
3. Capacity has some minor data modifications (such as "," to show thousands), so stripped from unnecessary commas and turned into integer data.


In [13]:

stadium_df['Country']=stadium_df['Country'].astype('category')
stadium_df['Pitch']=stadium_df['Pitch'].astype('category')

cols=['Team', 'Stadium', 'City']
stadium_df[cols]=stadium_df[cols].astype('string')

stadium_df['Capacity'].replace(',','', regex=True, inplace=True)
stadium_df['Capacity'] = pd.to_numeric(stadium_df['Capacity'],errors='coerce')
stadium_df['Capacity']=stadium_df['Capacity'].astype('int')

stadium_df

,Team,City,Country,Stadium,Capacity,Pitch
0,Wattenscheid 09,Bochum-Wattenscheid,Germany,Lohrheidestadion,16233,Natural
1,Hertha BSC,Berlín,Germany,Olympiastadion Berlin,76065,Natural
2,Unterhaching,Unterhaching,Germany,Sportpark Unterhaching,15053,Natural
3,Fortuna Köln,Cologne,Germany,Südstadion,14944,Natural
4,MSV Duisburg,Duisburgo,Germany,Schauinsland-Reisen-Arena,31514,Natural
...,...,...,...,...,...,...
539,Latina,Latina,Italy,Stadio Comunale Domenico Francioni,8000,Natural
540,Virtus Entella,Chiavari,Italy,Stadio Comunale Chiavari,4154,Césped Artificial
541,Nuova Cosenza,Cosenza,Italy,Stadio San Vito,24209,Natural
542,Pordenone,Pordenone,Italy,<NA>,3089,NaN


In [14]:

stadium_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544 entries, 0 to 543
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Team      544 non-null    string  
 1   City      544 non-null    string  
 2   Country   544 non-null    category
 3   Stadium   447 non-null    string  
 4   Capacity  544 non-null    int64   
 5   Pitch     447 non-null    category
dtypes: category(2), int64(1), string(3)
memory usage: 19.2 KB


HY

HYPOTHESES: what has higher influence of winning the match

1. Being the Home_Team
2. Plaing in Home_Country
3. Capacity of your Home_Statium (spectators-pressure)
4. Aggressivness as a playing style (check number of red/yellow cards per game)
5. 